# Face and eyes detection using Haar Cascades
https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html

In [1]:
import numpy as np
import cv2
import os

def get_model(model_dir):
    model = {
        'face': cv2.CascadeClassifier(os.path.join(model_dir, 'haarcascade_frontalface_default.xml')),
        'eye': cv2.CascadeClassifier(os.path.join(model_dir, 'haarcascade_eye.xml'))
    }
    return model

def face_eyes_detection(img, model):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = model['face'].detectMultiScale(gray, 1.3, 5)
    results = []
    for (x1,y1,w,h) in faces:
        x2, y2 = x1+w, y1+h
        cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),2)
        roi_gray = gray[y1:y2, x1:x2]
        roi_color = img[y1:y2, x1:x2]
        eyes = model['eye'].detectMultiScale(roi_gray)
        res_eyes = []
        for (ex1,ey1,ew,eh) in eyes:
            ex2, ey2 = ex1+ew, ey1+eh
            res_eyes.append((ex1, ey1,ex2, ey2))
            cv2.rectangle(roi_color,(ex1,ey1),(ex2,ey2),(0,255,0),2)
        results.append({
            'face': (x1, y1, x2, y2),
            'eyes': res_eyes
        })
    return (img, results)

In [2]:
cv2_data_dir = '/usr/local/lib/python3.7/site-packages/cv2/data'
model = get_model(model_dir=cv2_data_dir)

In [3]:
file_name = 'image1.jpg'
img = cv2.imread(os.path.join('data/input',file_name))
img, results = face_eyes_detection(img, model)
cv2.imwrite(os.path.join('data/detected',file_name), img)
results

[{'face': (408, 230, 612, 434),
  'eyes': [(81, 45, 140, 104), (21, 66, 67, 112)]},
 {'face': (733, 76, 921, 264), 'eyes': [(99, 43, 147, 91), (35, 53, 81, 99)]}]

## Face and eyes detection on video

In [16]:
def face_eyes_detection_on_video(input_video_path, output_video_path, model, params):
    inpVideo = cv2.VideoCapture(input_video_path)
    fps = int(inpVideo.get(cv2.CAP_PROP_FPS))
    width = int(inpVideo.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(inpVideo.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"\tfps={fps}, width={width}, height={height}")
    outVideo = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'H264'), fps, (width, height))
    start_id = int(params.get('start_sec',0) * fps)
    end_id = int(params.get('end_sec',0) * fps)
    print(f'start_id = {start_id}, end_id = {end_id}')
    results = []
    frame_id = 0
    ok, img = inpVideo.read()
    print("\t", end='')
    while ok:
        if frame_id >= start_id:
            img, res = face_eyes_detection(img, model)
            results.append(res)
            outVideo.write(img)
            if frame_id % fps == 0:
                print(".", end='')
        frame_id += 1
        if end_id != 0 and frame_id > end_id:
            break
        ok, img = inpVideo.read()
    outVideo.release()
    inpVideo.release()
    print("done.")
    return results

import youtube_dl
import re

def find_files(dirPath, pattern):
    files = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]
    ptrn = re.compile(pattern)
    files = list(filter(lambda f: ptrn.match(f), files))
    return [os.path.join(dirPath, f) for f in files]

def download_from_youtube(url, destDir):
    id_ = url.split('/')[-1]
    ydl_opts = {
        'outtmpl': destDir + '/%(id)s.%(ext)s'
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    files = find_files(destDir, '{}\..*'.format(id_))
    return files[0] if files else None

In [17]:
#url = 'https://youtu.be/EtBy7_rAeAI'
url = 'https://youtu.be/copMXgH3Z3E'
download_from_youtube(url, 'data/input')

[youtube] copMXgH3Z3E: Downloading webpage
[youtube] copMXgH3Z3E: Downloading video info webpage
[download] data/input/copMXgH3Z3E.mp4 has already been downloaded and merged


'data/input/copMXgH3Z3E.mp4'

In [18]:
video_file = 'copMXgH3Z3E.mp4'
results = face_eyes_detection_on_video(
    input_video_path  = os.path.join('data/input', video_file), 
    output_video_path = os.path.join('data/detected', video_file), 
    model = model,
    params = {
        'start_sec': 90,
        'end_sec': 90 + 30
    }
)
results

	fps=25, width=1920, height=1080
start_id = 2250, end_id = 3000
	...............................done.


[[{'face': (1321, 229, 1581, 489),
   'eyes': [(126, 76, 180, 130), (66, 78, 110, 122)]}],
 [{'face': (1328, 234, 1567, 473),
   'eyes': [(154, 202, 176, 224), (127, 70, 180, 123), (56, 65, 113, 122)]}],
 [{'face': (1335, 222, 1596, 483),
   'eyes': [(128, 79, 184, 135), (154, 211, 178, 235), (52, 74, 111, 133)]}],
 [{'face': (1325, 217, 1591, 483),
   'eyes': [(144, 82, 202, 140), (69, 79, 125, 135)]}],
 [{'face': (1324, 211, 1602, 489),
   'eyes': [(75, 84, 130, 139), (153, 87, 212, 146)]}],
 [{'face': (1334, 210, 1616, 492),
   'eyes': [(67, 83, 125, 141), (148, 87, 206, 145)]}],
 [{'face': (1345, 209, 1624, 488),
   'eyes': [(121, 173, 146, 198), (58, 84, 116, 142), (141, 89, 199, 147)]},
  {'face': (533, 207, 771, 445),
   'eyes': [(36, 45, 124, 133), (125, 72, 169, 116), (37, 67, 99, 129)]}],
 [{'face': (1349, 214, 1618, 483),
   'eyes': [(56, 77, 115, 136), (123, 167, 146, 190), (141, 83, 198, 140)]},
  {'face': (517, 209, 758, 450),
   'eyes': [(125, 63, 175, 113), (37, 61, 108